<font size=+2>**Dengue Time Series Example**</font>

- PyCaret

**More details in references**

- https://pycaret.gitbook.io/docs/get-started/quickstart#time-series

PyCaret เป็น library สำเร็จรูปที่สามารถใช้ทดสอบ หรือทำการเปรียบเทียบโมเดล machine learning หลายๆตัว ได้พร้อมๆกัน ช่วยให้ประหยัดเวลาในการหาโมเดลที่เหมาะสมกับชุดข้อมูลของเรา

ปล. ลงรอบแรกเพียงครั้งเดียว

In [ ]:
# install full version
!pip install 'pycaret[full]'

In [2]:
import pandas as pd

# Load Data

เนื่องจากข้อมูล Dengue มีการแบ่งไฟล์ออก features และ labels ออกจากกันจึงต้องนำมารวมกันเป็น dataframe เดียวก่อน จึงจะใช้ library PyCaret ได้

In [3]:
case = pd.read_csv("../r506_pipeline/data/df2.csv", encoding="cp874" )
temp = pd.read_csv("../temp_pipeline/data/dataset/temp.csv", encoding="cp874")
rain = pd.read_csv("../rain_pipeline/data/dataset/rain.csv", encoding="cp874")
press = pd.read_csv("../pressure_pipeline/data/dataset/press.csv", encoding="cp874")
humidity = pd.read_csv("../humidity_pipeline/data/dataset/humidity.csv", encoding="cp874")
# Merge the features and labels data
dengue_df = case.merge(temp,on='date',how='left')
dengue_df = dengue_df.merge(press,on='date',how='left')
dengue_df = dengue_df.merge(humidity,on='date',how='left')
dengue_df = dengue_df.merge(rain,on='date',how='left')
dengue_df.info()
# dengue_df = pd.merge(dengue_x, dengue_y, on=['city', 'year', 'weekofyear'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        232 non-null    object 
 1   total_case  232 non-null    int64  
 2   temp        232 non-null    float64
 3   press       224 non-null    float64
 4   humidity    232 non-null    float64
 5   rain        232 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 11.0+ KB


# Data Description

- city : เมือง
- year : ปี
- week of year : สัปดาห์ของปี
- week_start_date : วันที่เริ่มต้นรายสัปดาห์
- ndvi: ดัชนี NDVI
- reanalysis: ค่าสภาพอากาศ
- station: ค่าจากสถานีตรวจวัด

In [ ]:
dengue_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1456 entries, 0 to 1455
Data columns (total 25 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   city                                   1456 non-null   object 
 1   year                                   1456 non-null   int64  
 2   weekofyear                             1456 non-null   int64  
 3   week_start_date                        1456 non-null   object 
 4   ndvi_ne                                1262 non-null   float64
 5   ndvi_nw                                1404 non-null   float64
 6   ndvi_se                                1434 non-null   float64
 7   ndvi_sw                                1434 non-null   float64
 8   precipitation_amt_mm                   1443 non-null   float64
 9   reanalysis_air_temp_k                  1446 non-null   float64
 10  reanalysis_avg_temp_k                  1446 non-null   float64
 11  rean

เนื่องจากข้อมูลมี city 2 ประเภท คือ sj และ iq (ในที่นี้เลือกเมือง sj ในการทำโมเดล)


In [ ]:
# Select only the data for San Juan
dengue_df = dengue_df[dengue_df['city'] == 'sj']

ทำการ drop ข้อมูลที่ไม่ต้องการนำเข้าโมเดลออก แปลง week_start_date ให้เป็นประเภท datetime แล้วกำหนดให้เป็น index (ขั้นตอนเหมือนบทเรียน)

In [ ]:
dengue_df.drop(['city','year','weekofyear'], axis=1, inplace=True)
dengue_df['week_start_date'] = pd.to_datetime(dengue_df['week_start_date'])
dengue_df.set_index('week_start_date', inplace=True)

ตัวอย่างข้อมูล Dengue

In [4]:
dengue_df

,date,total_case,temp,press,humidity,rain
0,2019-01-07,30,27.282259,1005.820689,85.897126,7.777041
1,2019-01-14,18,27.177412,1005.009584,81.642037,4.164286
2,2019-01-21,18,26.849482,1007.035567,79.545775,0.621429
3,2019-01-28,23,27.270083,1006.782040,83.890311,5.496281
4,2019-02-04,28,27.903425,1008.452154,78.378565,0.078027
...,...,...,...,...,...,...
227,2023-12-01,28,26.863721,1007.435911,89.920629,5.500238
228,2023-12-08,24,26.813490,1007.187055,89.482885,32.002381
229,2023-12-15,18,27.116654,1008.151154,88.796577,6.510595
230,2023-12-22,11,25.604502,1010.282544,94.228968,59.655238


# Resample & Imputation

ทำการ resample ข้อมูลให้เริ่มต้นสัปดาห์เหมือนกันเป็นวันจันทร์ (W-MON) แล้วทำการ interpolate ข้อมูลที่หายไปด้วยวิธีการ linear (ขั้นตอนเหมือนบทเรียน)

In [ ]:
dengue_df = dengue_df.resample('W-MON').mean().interpolate(method='linear')

# Setup

ทำการ setup ข้อมูลเบื้องต้นก่อนการเข้าโมเดล (ขั้นตอนนี้อาจมี error เกิดขึ้นหากการเตรียมข้อมูลก่อนหน้ามีความผิดพลาด เช่น มี NaN อยู่ในข้อมูล หรือ ข้อมูลวันที่ยังไม่ได้ถูกแปลงเป็น datetime และถูกตั้งเป็น index เป็นต้น)

- target : column ที่จะทำนาย (ไม่ต้องระบุ หากข้อมูลมีแค่ column เดียว)
- fh : forecast horizon (จำนวน timestep ที่จะทำนาย) (จะถูกแบ่งเป็น test set ไว้สำหรับทดสอบโมเดล)
- fold : จำนวน K-Fold validation ที่ต้องการพับเพื่อทดสอบ
- session_id : เทียบเท่ากับ random_state ใช้สำหรับกำหนด seed ในการแบ่งข้อมูลเพื่อให้ได้ผลลัพธ์เดิมในทุกครั้งที่ทำการรัน

In [9]:
from pycaret.time_series import *
s = setup(dengue_df, target='total_cases', fh = 4, fold = 5, session_id = 123)

ModuleNotFoundError: No module named 'pycaret'

: 

# Compare Models

คำสั่ง compare_models จะทำการไล่เทรนชุดข้อมูลเรากับโมเดลหลากหลายประเภทที่เป็นไปได้ทั้งหมด แล้วแสดงผลลัพธ์ออกมาในรูปแบบของตาราง Forcast Error Metrics (ค่าผลลัพธ์ที่ดีที่สุดจะถูก highlight กำกับไว้)

In [8]:
best = compare_models()

NameError: name 'compare_models' is not defined

# Analyze Model

### Out-of-Sample Forecasting

In [ ]:
plot_model(best, plot = 'forecast', data_kwargs = {'fh' : 24})

### Residual Diagnostics

หมายเหตุ: บางครั้งอาจจะ plot ไม่ได้ เนื่องจากประเภทของโมเดลที่ใช้ไม่สามารถคำนวณออกมาได้

In [ ]:
plot_model(best, plot = 'diagnostics')

In sample predictions has not been implemented for this estimator of type 'BaseCdsDtForecaster' in `sktime`. When this is implemented, it will be enabled by default in pycaret.
In sample predictions has not been implemented for this estimator of type 'BaseCdsDtForecaster' in `sktime`. When this is implemented, it will be enabled by default in pycaret.


In [ ]:
plot_model(best, plot = 'insample')

In sample predictions has not been implemented for this estimator of type 'BaseCdsDtForecaster' in `sktime`. When this is implemented, it will be enabled by default in pycaret.


# Predictions

กำหนดค่า fh เพื่อกำหนดกรอบของการทำนาย (forecast horizon)

In [ ]:
final_best = finalize_model(best)
predict_model(best, fh = 24)

,y_pred
2008-04-01/2008-04-07,1.2782
2008-04-08/2008-04-14,1.3540
2008-04-15/2008-04-21,0.2694
2008-04-22/2008-04-28,0.6548
2008-04-29/2008-05-05,4.2219
2008-05-06/2008-05-12,5.7474
2008-05-13/2008-05-19,6.6951
2008-05-20/2008-05-26,8.6485
2008-05-27/2008-06-02,8.8032
2008-06-03/2008-06-09,9.0643


# Save Model

บันทึกโมเดลที่ดีที่สุดสำหรับเก็บไว้ใช้งาน

In [ ]:
# save_model(final_best, 'my_final_best_model')

# Load Model

การโหลดโมเดลที่บันทึกไว้ขึ้นมาใช้งาน

In [ ]:
# loaded_model = load_model('my_final_best_model')
# print(loaded_model)

<font color='red'> หมายเหตุ: ควรมีการกรองข้อมูล และตรวจสอบข้อมูลก่อนการนำเข้าโมเดลตามขั้นตอนในบทเรียน เนื่องจาก jupyter notebook นี้เป็นเพียงตัวอย่างการใช้งาน PyCaret เท่านั้น !</font>